<a href="https://colab.research.google.com/github/antonpetkoff/identifier-suggestion/blob/master/notebooks/00-colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# import utilities
import os
import shutil
from subprocess import check_output

In [2]:
!nvidia-smi

Fri Jun  5 09:38:11 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.82       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P8    10W /  70W |      0MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [4]:
%env WORKSPACE_DIR=/content/gdrive/My Drive/src

# TODO: how can one read an environment variable?!?
%cd '/content/gdrive/My Drive/src'

env: WORKSPACE_DIR=/content/gdrive/My Drive/src
/content/gdrive/My Drive/src


In [0]:
timestamp = check_output(['date', '-u', "+%Y-%m-%dT%H-%M-%S"]).decode('utf-8').strip()

os.environ['PROJECT_DIR'] = os.path.join(
    os.environ['WORKSPACE_DIR'],
    f'identifier-suggestion-{timestamp}',
)

In [6]:
!git clone https://github.com/antonpetkoff/identifier-suggestion.git --depth 1 "${PROJECT_DIR}"

Cloning into '/content/gdrive/My Drive/src/identifier-suggestion-2020-06-05T09-38-13'...
remote: Enumerating objects: 83, done.
remote: Counting objects: 100% (83/83), done.
remote: Compressing objects: 100% (76/76), done.
remote: Total 83 (delta 0), reused 46 (delta 0), pack-reused 0
Unpacking objects: 100% (83/83), done.


In [0]:
os.chdir(os.environ['PROJECT_DIR'])

In [8]:
!pwd
!ls -l

/content/gdrive/My Drive/src/identifier-suggestion-2020-06-05T09-38-13
total 60
-rw-------  1 root root 35149 Jun  5 09:38 LICENSE
drwx------  2 root root  4096 Jun  5 09:38 notebooks
-rw-------  1 root root  9130 Jun  5 09:38 README.md
drwx------  2 root root  4096 Jun  5 09:38 requirements
drwx------ 12 root root  4096 Jun  5 09:38 src
drwx------  3 root root  4096 Jun  5 09:38 vscode-extension


In [9]:
%tensorflow_version 2.x

import tensorflow
print(tensorflow.__version__)

2.2.0


In [10]:
# Google Colab has standard libraries like numpy, pandas, matplotlib and TF (of course) pre-installed
!pip install -r requirements/colab.txt

In [11]:
# provide secrets to the project, e.g. access to wandb
shutil.copy(
    os.path.join(os.environ['WORKSPACE_DIR'], 'secrets/.env'),
    os.environ['PROJECT_DIR']
)

'/content/gdrive/My Drive/src/identifier-suggestion-2020-06-05T09-38-13/.env'

In [0]:
# monkey-patch (mock) os.symlink to be a noop, because wandb.save() uses it, but it is not supported by Google Colab Notebooks
os.symlink = lambda *x: print('Executing mocked noop symlink with arguments', x)

In [13]:
from argparse import Namespace
from src.pipelines.baseline import run

params = {
  'file_data_raw': '../data/interim/method-names-rich/elasticsearch.csv',
  'file_checkpoint_dir': 'models/checkpoints/baseline/',
  'dir_preprocessed_data': '../data/interim/preprocessed/',
  'max_input_length': 200,
  'max_output_length': 8,
  'input_vocab_size': 20000,
  'input_embedding_dim': 64,
  'output_vocab_size': 6000,
  'output_embedding_dim': 64,
  'latent_dim': 256,
  'learning_rate': 0.0001,
  'epochs': 25,
  'batch_size': 512,
  'random_seed': 1,
  'eval_averaging': 'macro'
}

run(Namespace(**params))

Initializing logger


wandb: Wandb version 0.8.36 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


Experiment parameters:  Namespace(batch_size=512, dir_preprocessed_data='../data/interim/preprocessed/', epochs=25, eval_averaging='macro', file_checkpoint_dir='models/checkpoints/baseline/', file_data_raw='../data/interim/method-names-rich/elasticsearch.csv', input_embedding_dim=64, input_vocab_size=20000, latent_dim=256, learning_rate=0.0001, max_input_length=200, max_output_length=8, output_embedding_dim=64, output_vocab_size=6000, random_seed=1)
Loading preprocessed files...
Loaded input vocabulary.
Loaded output vocabulary.
Loaded preprocessed files.
Model: "encoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
EncoderEmbedding (Embedding) multiple                  1280000   
_________________________________________________________________
EncoderLSTM (LSTM)           multiple                  328704    
Total params: 1,608,704
Trainable params: 1,608,704
Non-trainable params: 0
_______________

wandb: Visualizing heatmap.


epoch 1 evaluation time: 5.438562393188477 sec
epoch evaluation:  {'epoch': 1, 'epoch_test_loss': <tf.Tensor: shape=(), dtype=float32, numpy=3.0431302>, 'epoch_test_sparse_categorical_accuracy': <tf.Tensor: shape=(), dtype=float32, numpy=0.13613209>, 'epoch_test_precision': <tf.Tensor: shape=(), dtype=float32, numpy=2.2688682e-05>, 'epoch_test_recall': <tf.Tensor: shape=(), dtype=float32, numpy=0.00016666666>, 'epoch_test_f1': <tf.Tensor: shape=(), dtype=float32, numpy=3.994022e-05>}


...

epoch 24 saved checkpoint: models/checkpoints/baseline/ckpt-9
epoch 25 - batch 10 - loss 1.8116103410720825 - precision 0.024563370272517204 - recall 0.021997570991516113 - f1 0.018564807251095772 - sparse_categorical_accuracy 0.2416294664144516
epoch 25 - batch 20 - loss 1.8441451787948608 - precision 0.028697019442915916 - recall 0.02444620430469513 - f1 0.02108096331357956 - sparse_categorical_accuracy 0.24362444877624512
epoch 25 - batch 30 - loss 1.8815268278121948 - precision 0.03161069378256798 - recall 0.025292091071605682 - f1 0.022136835381388664 - sparse_categorical_accuracy 0.24435454607009888
epoch 25 - batch 40 - loss 1.8673632144927979 - precision 0.033496253192424774 - recall 0.025618279352784157 - f1 0.022913891822099686 - sparse_categorical_accuracy 0.2450823038816452
epoch 25 - batch 50 - loss 1.8937957286834717 - precision 0.033888962119817734 - recall 0.026006018742918968 - f1 0.023259956389665604 - sparse_categorical_accuracy 0.2451171875
epoch 25 - batch 60 - los

wandb: Visualizing heatmap.


epoch 25 evaluation time: 5.6209986209869385 sec
epoch evaluation:  {'epoch': 25, 'epoch_test_loss': <tf.Tensor: shape=(), dtype=float32, numpy=2.0465825>, 'epoch_test_sparse_categorical_accuracy': <tf.Tensor: shape=(), dtype=float32, numpy=0.23766598>, 'epoch_test_precision': <tf.Tensor: shape=(), dtype=float32, numpy=0.031478256>, 'epoch_test_recall': <tf.Tensor: shape=(), dtype=float32, numpy=0.025963198>, 'epoch_test_f1': <tf.Tensor: shape=(), dtype=float32, numpy=0.022247516>}


wandb: Visualizing heatmap.
wandb: Visualizing heatmap.
wandb: Visualizing heatmap.
wandb: Visualizing heatmap.
wandb: Visualizing heatmap.
wandb: Visualizing heatmap.
wandb: Visualizing heatmap.
wandb: Visualizing heatmap.
wandb: Visualizing heatmap.


Saving checkpoint...
Saved checkpoint: models/checkpoints/baseline/ckpt-10
Done saving model
